In [9]:
from selenium import webdriver
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
def web_scrape(url):
    options = Options()
    options.add_argument("--enable-performance-logging")
    options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    log = driver.get_log("performance")
    driver.quit()
    
    return log

In [12]:
url = "https://www.oddsportal.com/football/belgium/jupiler-pro-league/"
scraped = web_scrape(url)

In [13]:
with open("oddsportal_network_dump.txt", 'w') as f:
    for i in scraped:
        test = i['message']
        json_obj = json.loads(test)
        json_formatted = json.dumps(json_obj, indent=2)
        print(json_formatted)
        f.write(json_formatted)
    f.close()

{
  "message": {
    "method": "Page.frameStartedLoading",
    "params": {
      "frameId": "5ED1FD6D599054FB640E20711CAAA5AC"
    }
  },
  "webview": "5ED1FD6D599054FB640E20711CAAA5AC"
}
{
  "message": {
    "method": "Network.requestWillBeSent",
    "params": {
      "documentURL": "https://www.oddsportal.com/football/belgium/jupiler-pro-league/",
      "frameId": "5ED1FD6D599054FB640E20711CAAA5AC",
      "hasUserGesture": false,
      "initiator": {
        "type": "other"
      },
      "loaderId": "90310839E674EB16810FC281125366EE",
      "redirectHasExtraInfo": false,
      "request": {
        "headers": {
          "Upgrade-Insecure-Requests": "1",
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
          "sec-ch-ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
          "sec-ch-ua-mobile": "?0",
          "sec-ch-ua-platform": "\"Windows\""
        },